# Stock Market Tracker - Latest Market Movers Dashboard

This notebook demonstrates an autonomous research agent that:
- Fetches latest stock market news and data
- Identifies stocks that rose and fell today
- Creates a beautiful HTML dashboard with market analysis

## Requirements

1. **API Keys** (set in `.env` file):
   - `ANTHROPIC_API_KEY` - Get from https://console.anthropic.com/

2. **MCP Servers** (install if needed):
   ```bash
   # Fetch server for web research
   pip install mcp-server-fetch
   
   # Filesystem server for saving HTML
   npm install -g @modelcontextprotocol/server-filesystem
   ```

3. **Python Packages**:
   ```bash
   pip install 'aisuite[anthropic,mcp]' python-dotenv
   ```

## Setup

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import aisuite as ai
from aisuite.mcp import MCPClient

# Load environment variables
load_dotenv()

# Verify required API keys
if not os.getenv("ANTHROPIC_API_KEY"):
    raise ValueError(
        "Missing ANTHROPIC_API_KEY\n"
        "Please add it to your .env file"
    )

print("✓ Environment configured")
print(f"✓ Working directory: {os.getcwd()}")
print(f"✓ Today's date: {datetime.now().strftime('%Y-%m-%d')}")
print("\nReady to track the markets!")

✓ Environment configured
✓ Working directory: /Users/rohit/fleet/leclerc/aisuite-prs/aisuite-main/aisuite/examples/agents
✓ Today's date: 2025-11-10

Ready to track the markets!


## Initialize MCP Servers

In [2]:
# Fetch MCP Server - for web research
fetch_mcp = MCPClient(
    command="uvx",
    args=["mcp-server-fetch"],
    name="fetch"
)

# Filesystem MCP Server - for saving HTML files
filesystem_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()],
    name="filesystem"
)

print("✓ MCP servers initialized")
print(f"\nAvailable tools:")
print(f"  - Fetch: {[tool['name'] for tool in fetch_mcp.list_tools()]}")
print(f"  - Filesystem: {[tool['name'] for tool in filesystem_mcp.list_tools()]}")

✓ MCP servers initialized

Available tools:
  - Fetch: ['fetch']
  - Filesystem: ['read_file', 'read_text_file', 'read_media_file', 'read_multiple_files', 'write_file', 'edit_file', 'create_directory', 'list_directory', 'list_directory_with_sizes', 'directory_tree', 'move_file', 'search_files', 'get_file_info', 'list_allowed_directories']


## Generate Market Movers Dashboard

Watch the AI agent:
1. Fetch latest stock market news and data
2. Identify top gainers and losers
3. Create a comprehensive HTML dashboard
4. **Save the HTML file FIRST** (critical!)
5. Provide summary

In [3]:
# Initialize aisuite client
client = ai.Client()

# Combine all available tools
all_tools = fetch_mcp.get_callable_tools() + filesystem_mcp.get_callable_tools()

print("📈 Fetching latest stock market data...\n")
print("=" * 60)

# Create comprehensive research prompt
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{
        "role": "user",
        "content": f"""Create a stock market movers dashboard for today ({datetime.now().strftime('%Y-%m-%d')}).

**⚠️ CRITICAL EXECUTION RULES - READ FIRST:**
1. DO NOT provide ANY text responses while executing tools
2. DO NOT say things like "Now let me...", "I'll create...", or "Let me gather..."
3. Execute ALL tools SILENTLY - fetch URLs, write files, etc.
4. Your FIRST and ONLY text response must come AFTER the HTML file is successfully written
5. If you respond with text before calling write_file, the tool execution loop will STOP and the file will NOT be created!

**REMEMBER**: Research → Write HTML → THEN (and only then) respond with summary!

---

Steps to execute SILENTLY (no text responses until step 6):

1. **Fetch latest market data** from these sources:
   - Yahoo Finance market movers: https://finance.yahoo.com/markets/stocks/gainers/
   - MarketWatch: https://www.marketwatch.com/
   - Or any other reliable financial news source
   
   Extract:
   - Top 3 stocks that rose today (gainers)
   - For each stock: ticker symbol, company name, price, % change
   - Major market indices (S&P 500, Dow, NASDAQ) if available
   - Key market news/headlines

2. **Create a beautiful HTML dashboard** with:
   
   **Header Section**:
   - Title: "Stock Market Movers - [Today's Date]"
   - Market sentiment indicator (bullish/bearish/neutral)
   
   **Top Gainers Section**:
   - Table or cards showing:
     * Company name and ticker
     * Current price
     * % change (in green)
     * Brief reason for rise if known
      
   **Market News Section**:
   - 3-5 key headlines impacting the market
   
   **Footer**:
   - Data sources cited
   - Timestamp
   - Disclaimer: "This is for informational purposes only, not financial advice"

4. **Styling requirements**:
   - Professional financial dashboard aesthetic
   - Color coding: GREEN for gains, RED for losses
   - Clean table or card-based layout
   - Responsive design
   - Modern typography
   - Use shadows and borders for visual separation.
 
5. **SAVE HTML FILE (still no text response yet!)**:
   - Use write_file to save as 'stock_market_dashboard.html'
   - Wait for confirmation that write succeeded
   
6. **NOW you can respond with text** - provide a summary with:
   - Confirmation that the HTML file was created
   - Number of stocks analyzed
   - Top 3 gainers with % changes
   - Optionally, Top 3 losers with % changes
   - Overall market sentiment

Make it look like a professional Bloomberg/Yahoo Finance dashboard!"""
    }],
    tools=all_tools,
    max_turns=30
)

print("\n" + "=" * 60)
print("✓ STOCK MARKET DASHBOARD COMPLETE!")
print("=" * 60)
print(f"\n{response.choices[0].message.content}")

📈 Fetching latest stock market data...


✓ STOCK MARKET DASHBOARD COMPLETE!

The stock market dashboard for November 10, 2025, has been successfully created and saved as 'stock_market_dashboard.html'. Here's a summary of the dashboard:

- **Number of Stocks Analyzed**: Multiple stocks, focusing on the top 3 gainers.
- **Top 3 Gainers**:
  1. **Cogent Biosciences, Inc. (COGT)**: Price $32.46, Change +119.03%
  2. **Navitas Semiconductor Corporation (NVTS)**: Price $9.60, Change +22.45%
  3. **Opendoor Technologies Inc. (OPEN)**: Price $7.99, Change +21.77%
- **Overall Market Sentiment**: Bullish
- **Key Market News**:
  - Sony raises profit forecast after earnings beat
  - U.S. markets rally in anticipation of FOMC meeting outcomes
  - European markets gain on strong manufacturing data

The dashboard provides a clean, professional, and responsive layout suitable for financial analysis.


## Verify File Was Created

In [4]:
filename = "stock_market_dashboard.html"

if os.path.exists(filename):
    file_size = os.path.getsize(filename)
    print(f"✅ File created successfully!")
    print(f"   - Filename: {filename}")
    print(f"   - Size: {file_size:,} bytes")
    print(f"   - Location: {os.path.abspath(filename)}")
else:
    print(f"❌ ERROR: File was not created!")
    print(f"   Expected: {filename}")
    print(f"\n   The agent may have stopped before writing the file.")
    print(f"   Try re-running with a higher max_turns value.")

✅ File created successfully!
   - Filename: stock_market_dashboard.html
   - Size: 4,231 bytes
   - Location: /Users/rohit/fleet/leclerc/aisuite-prs/aisuite-main/aisuite/examples/agents/stock_market_dashboard.html


## View the Generated Dashboard

In [5]:
from IPython.display import IFrame, display

if os.path.exists(filename):
    display(IFrame(src=filename, width=900, height=800))
    print(f"\n💡 Tip: Open '{filename}' in your browser for full-screen viewing!")
else:
    print(f"⚠️  File not found: {filename}")
    print("The dashboard was not created. Check the error above.")


💡 Tip: Open 'stock_market_dashboard.html' in your browser for full-screen viewing!


## Optional: View Tool Execution History

See what sources the agent consulted and when it wrote the file:

In [6]:
print("Tool Execution History:\n")
print("=" * 60)

fetch_count = 0
write_count = 0

for i, msg in enumerate(response.choices[0].intermediate_messages, 1):
    if msg.role == "assistant" and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tool_call in msg.tool_calls:
            if tool_call.function.name == 'fetch':
                fetch_count += 1
                args = json.loads(tool_call.function.arguments)
                url = args.get('url', 'N/A')
                print(f"\n[{i}] 🌐 Fetched: {url[:80]}...")
            elif tool_call.function.name == 'write_file':
                write_count += 1
                args = json.loads(tool_call.function.arguments)
                path = args.get('path', 'N/A')
                content_size = len(args.get('contents', ''))
                print(f"\n[{i}] 💾 WROTE FILE: {path} ({content_size:,} bytes)")

print(f"\n{'=' * 60}")
print(f"Total web fetches: {fetch_count}")
print(f"Total file writes: {write_count}")

if write_count == 0:
    print("\n⚠️  WARNING: No file writes detected!")
    print("   The agent did not call write_file.")
elif write_count == 1:
    print("\n✅ Perfect! File was written exactly once.")
else:
    print(f"\n⚠️  NOTE: File was written {write_count} times (may have been overwritten)")

Tool Execution History:


[1] 🌐 Fetched: https://finance.yahoo.com/markets/stocks/gainers/...

[1] 🌐 Fetched: https://www.marketwatch.com/...


AttributeError: 'dict' object has no attribute 'role'

## Market Refresh

Markets change throughout the day! Re-run the notebook to get fresh data.

In [ ]:
print(f"Dashboard generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n💡 Tip: Re-run the 'Generate Market Movers Dashboard' cell for updated data!")
print("\n⚠️  Disclaimer: This is for informational purposes only.")
print("   This is not financial advice. Always do your own research.")

## Cleanup

In [ ]:
# Close MCP connections
fetch_mcp.close()
filesystem_mcp.close()

print("✓ MCP servers closed")
if os.path.exists(filename):
    print(f"✓ Your dashboard is saved as: {filename}")
print("\n📊 Happy trading! (Remember: not financial advice!)")